# Data Wrangling

##### In this notebook, we'll be mainly using Yahoo Finance and the pandas library to fetch the Fortune 100 stock companies by market cap. The reason I am choosing the top 100 by market cap is that they provide more reliable data, are cleaner, and are less likely to have gaps in between. The top 100 in the Fortune 500 companies are heavily tech-dominant. This may not be reliable for others, but as an investor/trader, I invest in tech stocks 90% of the time, so this model is beneficial for me, as well as for lots of retail investors, since the Fortune 100 top companies are very popular overall.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import sys 
import os

### Fetch the top 100 and save it to a CSV

In [2]:
top100_df = pd.read_csv('../data/top100.csv')
top100_df.head(10)

,Symbol
0,NVDA
1,MSFT
2,AAPL
3,GOOG
4,AMZN
5,META
6,AVGO
7,TSLA
8,HOOD
9,JPM


##### The top 100 as of current time in August of 2025

In [3]:
top100_df = pd.read_csv("../data/top100.csv")
tickers = top100_df['Symbol'].tolist()
print("Number of tickers:", len(tickers)) 
print(tickers[:10])  # preview first 10

Number of tickers: 100
['NVDA', 'MSFT', 'AAPL', 'GOOG', 'AMZN', 'META', 'AVGO', 'TSLA', 'HOOD', 'JPM']


In [4]:
# !pip install yfinance --upgrade --no-cache-dir

In [5]:
import yfinance as yf # type: ignore

data = yf.download(
    tickers = tickers,
    start="2019-01-01",
    end="2024-12-31",
    group_by="ticker",
    auto_adjust=True,
    progress=True
)
 # Example to check if data is fetched correctly

[*********************100%***********************]  100 of 100 completed


In [6]:
data['AAPL'].sample(5)

Price,Open,High,Low,Close,Volume
Date,,,,,
2024-09-03,227.491260,227.939173,220.145443,221.738037,50190600
2021-01-14,127.450998,127.645874,125.463222,125.609390,90221800
2022-09-15,152.246703,152.827546,149.027531,150.002136,90481100
2021-03-16,122.664608,124.147908,121.708277,122.537750,115227900
2020-08-31,124.099333,127.426026,122.562438,125.519493,225702700


In [7]:
# More efficient reshaping with better error handling
if isinstance(data, pd.DataFrame) and hasattr(data.columns, 'levels'):
    print("Starting data reshaping...")
    
    # Reset index to get Date as a column
    data_reset = data.reset_index()
    
    # Get all unique tickers from MultiIndex columns
    all_tickers = data.columns.get_level_values(0).unique()
    print(f"Found {len(all_tickers)} tickers to process")
    
    # Initialize list to store reshaped data
    reshaped_data = []
    processed_count = 0
    
    # Process each ticker with progress tracking
    for i, ticker in enumerate(all_tickers):
        if i % 10 == 0:  # Print progress every 10 tickers
            print(f"Processing ticker {i+1}/{len(all_tickers)}: {ticker}")
        
        try:
            # Check if ticker columns exist and have data
            if (ticker, 'Open') not in data.columns:
                continue
                
            # Extract data for this ticker 
            ticker_data = pd.DataFrame()
            ticker_data['date'] = data_reset['Date']
            ticker_data['open'] = data_reset[(ticker, 'Open')]
            ticker_data['high'] = data_reset[(ticker, 'High')]
            ticker_data['low'] = data_reset[(ticker, 'Low')]
            ticker_data['close'] = data_reset[(ticker, 'Close')]
            ticker_data['volume'] = data_reset[(ticker, 'Volume')]
            
            # Remove rows with all NaN values (except date)
            ticker_data = ticker_data.dropna(subset=['open', 'high', 'low', 'close'], how='all')
            
            # Only add ticker column AFTER cleaning the data
            if len(ticker_data) > 0:
                ticker_data['ticker'] = ticker
                # Reorder columns to put ticker first
                ticker_data = ticker_data[['ticker', 'date', 'open', 'high', 'low', 'close', 'volume']]
                reshaped_data.append(ticker_data)
                processed_count += 1
            
        except Exception as e:
            print(f"Error processing {ticker}: {str(e)}")
            continue
    
    print(f"Successfully processed {processed_count} tickers")
    
    # Concatenate all ticker data
    if reshaped_data:
        fortune_100 = pd.concat(reshaped_data, ignore_index=True)
        
        # Sort by ticker and date
        fortune_100 = fortune_100.sort_values(['ticker', 'date']).reset_index(drop=True)
        
        print(f"✅ Reshaped data successfully!")
        print(f"📊 Shape: {fortune_100.shape}")
        print(f"🏢 Unique tickers: {fortune_100['ticker'].nunique()}")
        print(f"📅 Date range: {fortune_100['date'].min()} to {fortune_100['date'].max()}")

        cols_to_round = ['open', 'high', 'low', 'close']
        fortune_100[cols_to_round] = fortune_100[cols_to_round].round(2)

        # Save to CSV
        fortune_100.to_csv('formatted_stock_data.csv', index=False)
        print(f"\n💾 Data saved to 'formatted_stock_data.csv'")

    else:
        print("❌ No data was reshaped")
        
else:
    print("❌ Error: 'data' is not a proper MultiIndex DataFrame from yfinance")
    print(f"Data type: {type(data)}")
    if hasattr(data, 'columns'):
        print(f"Columns type: {type(data.columns)}")

Starting data reshaping...
Found 100 tickers to process
Processing ticker 1/100: GEV
Processing ticker 11/100: WELL
Processing ticker 21/100: TJX
Processing ticker 31/100: TMO
Processing ticker 41/100: PANW
Processing ticker 51/100: COP
Processing ticker 61/100: ABT
Processing ticker 71/100: PEP
Processing ticker 81/100: UNH
Processing ticker 91/100: GOOG
Successfully processed 100 tickers
✅ Reshaped data successfully!
📊 Shape: (147186, 7)
🏢 Unique tickers: 100
📅 Date range: 2019-01-02 00:00:00 to 2024-12-30 00:00:00

💾 Data saved to 'formatted_stock_data.csv'


## Data Reshaping Process

**Purpose**: Transform yfinance MultiIndex DataFrame into long format with columns: ticker, date, open, high, low, close, volume

**Steps:**
1. **Validation** - Check if data is proper MultiIndex DataFrame from yfinance
2. **Reset Index** - Convert Date from index to column 
3. **Extract Tickers** - Get all unique ticker symbols from MultiIndex columns
4. **Process Each Ticker**:
   - Extract OHLCV data for each ticker
   - Clean data by removing rows with all NaN values
   - Add ticker column and reorder columns
5. **Combine Data** - Concatenate all tickers into single DataFrame
6. **Sort & Save** - Sort by ticker and date, save to CSV

**Error Handling**: Skips tickers with missing data or processing errors

In [8]:
# Rename columns to uppercase
fortune_100 = fortune_100.rename(columns={
    'ticker': 'Ticker', 
    'date': 'Date', 
    'open': 'Open', 
    'high': 'High', 
    'low': 'Low', 
    'close': 'Close', 
    'volume': 'Volume'
})

fortune_100.head()

,Ticker,Date,Open,High,Low,Close,Volume
0,AAPL,2019-01-02,36.85,37.80,36.70,37.58,148158800.0
1,AAPL,2019-01-03,34.26,34.67,33.79,33.83,365248800.0
2,AAPL,2019-01-04,34.39,35.35,34.22,35.28,234428400.0
3,AAPL,2019-01-07,35.38,35.41,34.72,35.20,219111200.0
4,AAPL,2019-01-08,35.59,36.12,35.34,35.87,164101200.0


## Calculate log return features

This is essential for capturing the stock price return from 1,2,3 day and so on, it'll help us know how the stock is doing in short term vs something like 2 weeks ago. These features will help us capture insights on the trend how the stock is to peform.

- `1-day:`**(Target Variable Feature)** Grouped by ticker so for each ticker not mixed between stocks, this is will be our Target Variable Feature during preprocessing and training step since this is the feature that knows tells us how the stock performed toda. We will use this to predict how the stock price will be at after today. Shifting the data by 1 day to make it a Time Series, it uses the previous day forecasting data to predict for the next day. 

- `2-day`: This another lagged feature to add so the model can predict using the price range from multiple range.
- `3-day`: Another short term lagged feature to capture short term patterns the model may miss.
- `5-day:` Then we will have a extra feature to predict 5 day ahead into the future, this will not be the Target Variable only for extra feature to see if the model is able to predict for 5 days ahead.
- `10-day`: This is the final lagged feature to let the model know how the stock ticker was moving around 10 days, it goes back further back in time to learn some patterns.

In [9]:
# 1-day log return (target variable)
fortune_100['LogRet_1D'] = fortune_100.groupby('Ticker')['Close'].transform(lambda x: np.log(x / x.shift(1)))

# 2-day log return (feature)
fortune_100['LogRet_2D'] = fortune_100.groupby('Ticker')['Close'].transform(lambda x: np.log(x / x.shift(2)))

# 3-day log return (feature)
fortune_100['LogRet_3D'] = fortune_100.groupby('Ticker')['Close'].transform(lambda x: np.log(x / x.shift(3)))

# 5-day log return (feature)
fortune_100['LogRet_5D'] = fortune_100.groupby('Ticker')['Close'].transform(lambda x: np.log(x / x.shift(5)))

# 10-day log return (feature)
fortune_100['LogRet_10D'] = fortune_100.groupby('Ticker')['Close'].transform(lambda x: np.log(x / x.shift(10)))


fortune_100[["LogRet_1D", "LogRet_2D","LogRet_3D", "LogRet_5D", "LogRet_10D"]] = fortune_100[["LogRet_1D", "LogRet_2D","LogRet_3D", "LogRet_5D", "LogRet_10D"]].round(5)


In [10]:
fortune_100.head(10)

,Ticker,Date,Open,High,Low,Close,Volume,LogRet_1D,LogRet_2D,LogRet_3D,LogRet_5D,LogRet_10D
0,AAPL,2019-01-02,36.85,37.80,36.70,37.58,148158800.0,NaN,NaN,NaN,NaN,NaN
1,AAPL,2019-01-03,34.26,34.67,33.79,33.83,365248800.0,-0.10512,NaN,NaN,NaN,NaN
2,AAPL,2019-01-04,34.39,35.35,34.22,35.28,234428400.0,0.04197,-0.06316,NaN,NaN,NaN
3,AAPL,2019-01-07,35.38,35.41,34.72,35.20,219111200.0,-0.00227,0.03970,-0.06543,NaN,NaN
4,AAPL,2019-01-08,35.59,36.12,35.34,35.87,164101200.0,0.01886,0.01659,0.05855,NaN,NaN
5,AAPL,2019-01-09,36.00,36.77,35.60,36.48,180396400.0,0.01686,0.03572,0.03345,-0.02971,NaN
6,AAPL,2019-01-10,36.29,36.64,35.90,36.59,143122800.0,0.00301,0.01987,0.03873,0.07843,NaN
7,AAPL,2019-01-11,36.38,36.57,36.05,36.24,108092800.0,-0.00961,-0.00660,0.01026,0.02685,NaN
8,AAPL,2019-01-14,35.89,35.99,35.51,35.69,129756800.0,-0.01529,-0.02490,-0.02189,0.01382,NaN
9,AAPL,2019-01-15,35.75,36.50,35.70,36.42,114843600.0,0.02025,0.00495,-0.00466,0.01522,NaN


## Lagged Volume features

Volume spikes can signal increased interest or news. Sudden drops or rises in volume may precede volatility. Lagged features help the model learn this temporal insights.

`Volume (t-1)`: Create a lagged feature of 1 day shift to avoid data leakage.
`Volume (t-5)`: To capture vollume patterns if any news that may have drive trading numbers high or low.

In [11]:
# 1-day lagged volume
fortune_100['Volume_Lag1'] = fortune_100.groupby('Ticker')['Volume'].transform(lambda x: x.shift(1))

# 5-day lagged volume
fortune_100['Volume_Lag5'] = fortune_100.groupby('Ticker')['Volume'].transform(lambda x: x.shift(5))

fortune_100.head()

,Ticker,Date,Open,High,Low,Close,Volume,LogRet_1D,LogRet_2D,LogRet_3D,LogRet_5D,LogRet_10D,Volume_Lag1,Volume_Lag5
0,AAPL,2019-01-02,36.85,37.80,36.70,37.58,148158800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAPL,2019-01-03,34.26,34.67,33.79,33.83,365248800.0,-0.10512,NaN,NaN,NaN,NaN,148158800.0,NaN
2,AAPL,2019-01-04,34.39,35.35,34.22,35.28,234428400.0,0.04197,-0.06316,NaN,NaN,NaN,365248800.0,NaN
3,AAPL,2019-01-07,35.38,35.41,34.72,35.20,219111200.0,-0.00227,0.03970,-0.06543,NaN,NaN,234428400.0,NaN
4,AAPL,2019-01-08,35.59,36.12,35.34,35.87,164101200.0,0.01886,0.01659,0.05855,NaN,NaN,219111200.0,NaN


## Simple Moving Average (SMA) Features

Adding SMA features such as ( `SMA10`, `SMA20`, `SMA50`, `SMA200`) is a uselful for it'll capture different momentum trends in stock prices over differen time shifts.

- `SMA_10`: These are the short term averages which will help capture short term momentum and sudden movement reversals.

- `SMA_20`: This is another short term but a little longer than 10 days, it'll help capture short term for the last 20 days average price volatility.
- `SMA_50`: This is a medium term, it'll help capture the price trends over a a good amount of time, this helps with traders confirm the trend direction.
- `SMA_200`: This is long term average movement, this is to help identify if there will be a bull or bear market.


In [12]:
# SMA 10, SMA 20, SMA 50, SMA 100...

fortune_100['SMA_10'] = fortune_100.groupby('Ticker')['Close'].transform(lambda x: x.rolling(window=10, min_periods=1).mean())
fortune_100['SMA_20'] = fortune_100.groupby('Ticker')['Close'].transform(lambda x: x.rolling(window=20, min_periods=1).mean())
fortune_100['SMA_50'] = fortune_100.groupby('Ticker')['Close'].transform(lambda x: x.rolling(window=50, min_periods=1).mean())
fortune_100['SMA_200'] = fortune_100.groupby('Ticker')['Close'].transform(lambda x: x.rolling(window=200, min_periods=1).mean())

sma_cols_round = ['SMA_10', 'SMA_20', 'SMA_50', 'SMA_200']    
fortune_100[sma_cols_round] = fortune_100[sma_cols_round].round(2)

fortune_100[['Ticker','Date','Close','SMA_10', 'SMA_20', 'SMA_50', 'SMA_200']].head()

,Ticker,Date,Close,SMA_10,SMA_20,SMA_50,SMA_200
0,AAPL,2019-01-02,37.58,37.58,37.58,37.58,37.58
1,AAPL,2019-01-03,33.83,35.70,35.70,35.70,35.70
2,AAPL,2019-01-04,35.28,35.56,35.56,35.56,35.56
3,AAPL,2019-01-07,35.20,35.47,35.47,35.47,35.47
4,AAPL,2019-01-08,35.87,35.55,35.55,35.55,35.55


## Exponential Moving Average (EMA) Features

Unlike SMA that captures the movement price for the the past N number of days equally. EMA gives more weight to recent price changes. This is useful for the model to react faster to short-term movements compared to SMAs. It is able to capture suddent movement while still knowing the overall market for long term.

- `EMA_12`: Short-term trend, reacts quickly to recent price changes.

- `EMA_26`: Medium-term trend, has smooter trend levels vs EMA_12, often used as a trading strategy with EMA_12 in MACD.
- `EMA_50`: Mid-range trend, helps confirm the trend direction of the stock.
- `EMA_200`: Long-term trend, this similar to SMA can be used to confirm if the market is going to a bullish or bearish phase.

In [13]:
# EMA_12, EMA_26, EMA_50, EMA_200...

fortune_100['EMA_12'] = fortune_100.groupby('Ticker')['Close'].transform(lambda x: x.ewm(span=12, adjust=False).mean())
fortune_100['EMA_26'] = fortune_100.groupby('Ticker')['Close'].transform(lambda x: x.ewm(span=26, adjust=False).mean())
fortune_100['EMA_50'] = fortune_100.groupby('Ticker')['Close'].transform(lambda x: x.ewm(span=50, adjust=False).mean())
fortune_100['EMA_200'] = fortune_100.groupby('Ticker')['Close'].transform(lambda x: x.ewm(span=200, adjust=False).mean())

ema_cols_round = ['EMA_12', 'EMA_26', 'EMA_50', 'EMA_200']    
fortune_100[ema_cols_round] = fortune_100[ema_cols_round].round(2)

fortune_100[['Ticker','Date','Close','EMA_12', 'EMA_26', 'EMA_50', 'EMA_200']].head()

,Ticker,Date,Close,EMA_12,EMA_26,EMA_50,EMA_200
0,AAPL,2019-01-02,37.58,37.58,37.58,37.58,37.58
1,AAPL,2019-01-03,33.83,37.00,37.30,37.43,37.54
2,AAPL,2019-01-04,35.28,36.74,37.15,37.35,37.52
3,AAPL,2019-01-07,35.20,36.50,37.01,37.26,37.50
4,AAPL,2019-01-08,35.87,36.40,36.92,37.21,37.48


## Relative Strength Index (RSI) Feature

`RSI(14)` measures the recent momentum oscillator that shows how a stock is performing in the current market.

- Overbought too expensive for traders / investers to buy, RSI above 70.
- Oversold too cheap for traders / investors to sell, RSI below 30

The range is from 0 - 100, and these indication is used for traders to spot reversals or upward momentum.

In [14]:
def rsi_wilder(x: pd.Series, period: int = 14) -> pd.Series:
    # calculate daily price changes
    delta = x.diff()
    
    gain = (delta.where(delta > 0, 0)).fillna(0) # keep only the positives and make others 0
    loss = (-delta.where(delta < 0, 0)).fillna(0) # keep only the negatives and then negate to make it positive

    avg_gain = gain.ewm(alpha=1/period, adjust=False).mean() # get the gain to smooth it using exponential moving average
    avg_loss = loss.ewm(alpha=1/period, adjust=False).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

fortune_100["RSI_14"] = fortune_100.groupby("Ticker")["Close"].transform(lambda x: rsi_wilder(x, period=14))
fortune_100["RSI_14"] = fortune_100["RSI_14"].round(2)

fortune_100[["Ticker", "Date", "RSI_14"]].groupby("Ticker").head()

,Ticker,Date,RSI_14
0,AAPL,2019-01-02,NaN
1,AAPL,2019-01-03,0.00
2,AAPL,2019-01-04,29.40
3,AAPL,2019-01-07,28.89
4,AAPL,2019-01-08,38.43
...,...,...,...
145677,XOM,2019-01-02,NaN
145678,XOM,2019-01-03,0.00
145679,XOM,2019-01-04,71.75
145680,XOM,2019-01-07,74.63


## Calcuate Rolling Return Sum Features

Unlike the lagged log returns which shows us the exact point to point changes, rolling sums adds up all the daily returns inside the timeframe given, so it is able to capture momentum trends short and long term.

- `3_day`: A 3 day rolling sum will cpature the cumulative price movements from the past 3 days. Helps the model understand very short-term momentum.

- `5_day`: A 5 day rolling sum represnts the past trading weeks total return movement, this provides the model with short term trend.
- `10_day:` A 10 day rolling sum represents two total trading weeks of the total return, helps the model learn longer momentum patterns.

In [15]:
# Rolling Return Sum Features

# 3-day rolling return sum
fortune_100["RollRetSum_3"] = fortune_100.groupby("Ticker")["LogRet_1D"].transform(lambda x: x.rolling(3).sum())

# 5-day rolling return sum
fortune_100["RollRetSum_5"] = fortune_100.groupby("Ticker")["LogRet_1D"].transform(lambda x: x.rolling(5).sum())

# 10-day rolling return sum
fortune_100["RollRetSum_10"] = fortune_100.groupby("Ticker")["LogRet_1D"].transform(lambda x: x.rolling(10).sum())

fortune_100[["RollRetSum_3","RollRetSum_5","RollRetSum_10"]] = fortune_100[["RollRetSum_3","RollRetSum_5","RollRetSum_10"]].round(4)

fortune_100[["Ticker","Date","Close","RollRetSum_3","RollRetSum_5","RollRetSum_10"]].head()

,Ticker,Date,Close,RollRetSum_3,RollRetSum_5,RollRetSum_10
0,AAPL,2019-01-02,37.58,NaN,NaN,NaN
1,AAPL,2019-01-03,33.83,NaN,NaN,NaN
2,AAPL,2019-01-04,35.28,NaN,NaN,NaN
3,AAPL,2019-01-07,35.20,-0.0654,NaN,NaN
4,AAPL,2019-01-08,35.87,0.0586,NaN,NaN


## Rolling Volatility Feature

This feature calculates the price action of a stock that can go up or down rapidly, this measures how the stock has been moving recently such how choppy or smooth the return has been.

- High volatility means that the price of the stock has been moving up and down a lot day to day, a lot of swings
- Low volatility means that the price of the stock is moving steadily smoother not much swings.

Using this feature will let the model know exactly if the market has been trading smooth or not, it allows to measure sudden changes. 

In [16]:
fortune_100["Volatility_10D"] = fortune_100.groupby("Ticker")["LogRet_1D"].transform(lambda s: s.rolling(10).std(ddof=0))

fortune_100["Volatility_10D"] = fortune_100["Volatility_10D"].round(4)

fortune_100[['Ticker', 'Close', 'Date', 'Volatility_10D']].tail()

,Ticker,Close,Date,Volatility_10D
147181,XOM,103.43,2024-12-23,0.0075
147182,XOM,103.53,2024-12-24,0.0078
147183,XOM,103.62,2024-12-26,0.0080
147184,XOM,103.61,2024-12-27,0.0080
147185,XOM,102.91,2024-12-30,0.0080


## MACD (Moving Average Convergence Divergence) Feature

This is a momement indicator will let the model know if the stock is picking up momenetum up or down. It does it so by comparing two moving averages.

- Fast EMA (12) reacts quick to price changes
- Slow EMA (26) reacts slower to price changes
 
- `MACD_Line`: Calculates the Fast EMA - Slow EMA, short term buy trend is stronger is MACD > 0, bullish momentum, MACD < 0, bearish momentum.
- `Signal_Line`: A 9-day EMA of the MACD line, this is used as a trigger to buy or sell.
- `MACD_Hist`: The difference between the MACD line and signal line, shows the when momentum is shifting.

There are crossovers between the two lines they are known as convergence or divergence. When the singal line crosses above the MACD line it is known as a bearish crossover that the sellers are dominating, and when signal line crosses below the MACD line, it is seen as a bullish crossover which indicatest that the buyers are now dominating against the sellers.

In [17]:
def macd_calc(series, fast = 12, slow = 26, signal = 9):
    
    ema_fast = series.ewm(span=fast, adjust=False).mean()
    ema_slow = series.ewm(span=slow, adjust=False).mean()
    
    macd_line = ema_fast - ema_slow
    macd_signal = macd_line.ewm(span=signal, adjust=False).mean()
    macd_hist = macd_line - macd_signal
    
    return pd.DataFrame({
        "MACD_Line": macd_line,
        "MACD_Signal": macd_signal,
        "MACD_Hist": macd_hist
    })

# Use function to apply for 3 macd features
macd_df = fortune_100.groupby("Ticker")["Close"].apply(macd_calc).reset_index(level=0, drop=True)

# Add to main DF with shift(1) to avoid leakage
fortune_100[["MACD_Line","MACD_Signal","MACD_Hist"]] = macd_df

fortune_100[["MACD_Line","MACD_Signal","MACD_Hist"]] = fortune_100[["MACD_Line","MACD_Signal","MACD_Hist"]].round(3)

fortune_100[['Ticker', 'Date', 'MACD_Line', 'MACD_Signal', 'MACD_Hist']].head()

,Ticker,Date,MACD_Line,MACD_Signal,MACD_Hist
0,AAPL,2019-01-02,0.000,0.000,0.000
1,AAPL,2019-01-03,-0.299,-0.060,-0.239
2,AAPL,2019-01-04,-0.414,-0.131,-0.284
3,AAPL,2019-01-07,-0.506,-0.206,-0.301
4,AAPL,2019-01-08,-0.519,-0.269,-0.251


In [18]:
fortune_100.head()

,Ticker,Date,Open,High,Low,Close,Volume,LogRet_1D,LogRet_2D,LogRet_3D,...,EMA_50,EMA_200,RSI_14,RollRetSum_3,RollRetSum_5,RollRetSum_10,Volatility_10D,MACD_Line,MACD_Signal,MACD_Hist
0,AAPL,2019-01-02,36.85,37.80,36.70,37.58,148158800.0,NaN,NaN,NaN,...,37.58,37.58,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.000
1,AAPL,2019-01-03,34.26,34.67,33.79,33.83,365248800.0,-0.10512,NaN,NaN,...,37.43,37.54,0.00,NaN,NaN,NaN,NaN,-0.299,-0.060,-0.239
2,AAPL,2019-01-04,34.39,35.35,34.22,35.28,234428400.0,0.04197,-0.06316,NaN,...,37.35,37.52,29.40,NaN,NaN,NaN,NaN,-0.414,-0.131,-0.284
3,AAPL,2019-01-07,35.38,35.41,34.72,35.20,219111200.0,-0.00227,0.03970,-0.06543,...,37.26,37.50,28.89,-0.0654,NaN,NaN,NaN,-0.506,-0.206,-0.301
4,AAPL,2019-01-08,35.59,36.12,35.34,35.87,164101200.0,0.01886,0.01659,0.05855,...,37.21,37.48,38.43,0.0586,NaN,NaN,NaN,-0.519,-0.269,-0.251


# Target Variable 

Since the objective of this project is to predict future stock movement, we will be using regression version to predict the log return. 

Regression Version: 

![Log Return Formula](../images/log.png)

using the LogRet_1D which is the log return for today. Using shift(-1) this will use all the features such as MACD, RSI, EMA, etc, then train the model to predict the tomorrows return. 
- (`Target_Variable`): This will be the target variable by shifting it'll predict the next day which is can be all indicators on Day 2 which and the target variable return will be for Day 3. In other words it uses all the features of today's closing price and makes a log return for the next day which can be compared using the market when it closes. But in our case here we are comparing it using historical data.

In [19]:
fortune_100["Target_LogRet_1D"] = fortune_100.groupby("Ticker")["LogRet_1D"].shift(-1)
fortune_100["Target_LogRet_1D"] = fortune_100["Target_LogRet_1D"].round(2)

fortune_100[['Ticker', 'Date','Close', 'LogRet_1D', 'Target_LogRet_1D']].head(10)

,Ticker,Date,Close,LogRet_1D,Target_LogRet_1D
0,AAPL,2019-01-02,37.58,NaN,-0.11
1,AAPL,2019-01-03,33.83,-0.10512,0.04
2,AAPL,2019-01-04,35.28,0.04197,-0.00
3,AAPL,2019-01-07,35.20,-0.00227,0.02
4,AAPL,2019-01-08,35.87,0.01886,0.02
5,AAPL,2019-01-09,36.48,0.01686,0.00
6,AAPL,2019-01-10,36.59,0.00301,-0.01
7,AAPL,2019-01-11,36.24,-0.00961,-0.02
8,AAPL,2019-01-14,35.69,-0.01529,0.02
9,AAPL,2019-01-15,36.42,0.02025,0.01


In [20]:
fortune_100.to_csv("../data/fortune_100_cleaned.csv", index=False)